In [1]:
import requests

headers = {
    'accept': '*/*',
}

params = {
    'page': '1',
    'perPage': '10',
    'serviceKey': 'ruy8A/v/syRNrte7Dyj7znYCM1HtSahtkKc7ZHTQUu+9dzLXkgdmTkbKhSwUqNwY2ksAHgrKPg/eV2SGtl1N6A==',
}

response = requests.get(
    'https://api.odcloud.kr/api/15084167/v1/uddi:1c8b5454-bd4e-45db-98f7-fe94d71f271b',
    params=params,
    headers=headers,
)

In [2]:
response

<Response [200]>

In [4]:
response.json()

{'currentCount': 10,
 'data': [{'구분번호': 1310440,
   '글제목': '대통령 취임사(大統領就任辭)',
   '대통령': '이승만',
   '연설일자': '1948-07-24',
   '연설장소': '국내',
   '원문보기': 'http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/11/26/DOC/SRC/0104200911263997500039975011942.PDF'},
  {'구분번호': 1310441,
   '글제목': '민족이 원하는 길을 따를 결심, 국무총리 인준 부결에 대하여',
   '대통령': '이승만',
   '연설일자': '1948-07-29',
   '연설장소': '국내',
   '원문보기': 'http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/11/26/DOC/SRC/0104200911263997600039976011943.PDF'},
  {'구분번호': 1310442,
   '글제목': '미급점(未及點) 육성하라',
   '대통령': '이승만',
   '연설일자': '1948-08-09',
   '연설장소': '국내',
   '원문보기': 'http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/11/26/DOC/SRC/0104200911263997700039977011944.PDF'},
  {'구분번호': 1310443,
   '글제목': '전민족에게 충고함',
   '대통령': '이승만',
   '연설일자': '1948-08-12',
   '연설장소': '국내',
   '원문보기': 'http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/11/26/DOC/SRC/0104200911263997800039978011945.PDF'},
  {'구분번호': 1310445,
   '글제목': '함께 뭉쳐서 자강전진(自疆前進) 외모(外侮)막자 (해방 3주년 기념사)',
   '대통령': '이승만',

In [5]:
# https://stackoverflow.com/questions/45470964/python-extracting-text-from-webpage-pdf

In [60]:
import pandas as pd

# csv 파일 열기
df = pd.read_csv("president_speeches_sample100.csv")

In [61]:
df

,구분번호,대통령,글제목,연설일자,원문보기,연설장소
0,1310440,이승만,대통령 취임사(大統領就任辭),1948-07-24,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
1,1310441,이승만,"민족이 원하는 길을 따를 결심, 국무총리 인준 부결에 대하여",1948-07-29,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
2,1310442,이승만,미급점(未及點) 육성하라,1948-08-09,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
3,1310443,이승만,전민족에게 충고함,1948-08-12,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
4,1310445,이승만,함께 뭉쳐서 자강전진(自疆前進) 외모(外侮)막자 (해방 3주년 기념사),1948-08-15,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
...,...,...,...,...,...,...
95,1310536,이승만,토탄을 사용하라,1949-09-30,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
96,1310537,이승만,"공산당과 협의 불가, 인권보증에 결사투쟁",1949-10-07,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
97,1310538,이승만,한글사용에 관하여,1949-10-12,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
98,1310539,이승만,대한 적십자사 재조직 중앙대회에 전하는 훈사,1949-10-25,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내


In [63]:
import sqlite3
from pypdf import PdfReader
import requests
import io

connection = sqlite3.connect("president_speeches.db")
cursor = connection.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS president_speeches (
division_number INT,
president       TEXT,
title           TEXT,
date            TEXT,
pdf_url         TEXT,
location        TEXT,
speech_text     TEXT);
""")

connection.commit()

# csv 파일의 각 행을 순서대로 읽어 옴
for idx, row in df.iterrows():

    # 구분번호, 대통령, 글제목, 연설일자, 원문보기, 연설장소 를 변수에 저장
    print(row)
    division_number = row['구분번호']
    president = row['대통령']
    title = row['글제목']
    date = row['연설일자']
    pdf_url = row['원문보기']
    location = row['연설장소']
    
    r = requests.get(pdf_url)
    f = io.BytesIO(r.content)
    
    reader = PdfReader(f)
    page = reader.pages[0]
    # 연설원문을 변수에 저장
    speech_text = page.extract_text().replace("\n", "")

    # 데이터베이스에 저장
    cursor.execute("""INSERT INTO president_speeches (
    division_number, 
    president, 
    title, 
    date, 
    pdf_url, 
    location, 
    speech_text) VALUES (?, ?, ?, ?, ?, ?, ?);""", 
                   (division_number, president, title, date, pdf_url, location, speech_text))
    connection.commit()

# 데이터베이스 연결 종료
connection.close()


구분번호                                              1310440
대통령                                                   이승만
글제목                                      대통령 취임사(大統領就任辭) 
연설일자                                           1948-07-24
원문보기    http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...
연설장소                                                   국내
Name: 0, dtype: object
구분번호                                              1310441
대통령                                                   이승만
글제목                    민족이 원하는 길을 따를 결심, 국무총리 인준 부결에 대하여 
연설일자                                           1948-07-29
원문보기    http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...
연설장소                                                   국내
Name: 1, dtype: object
구분번호                                              1310442
대통령                                                   이승만
글제목                                        미급점(未及點) 육성하라 
연설일자                                           1948-08-09
원문보기    http://dams.pa.go.